# Twitter Sentiment Analysis - POC
---

## 5. Pre-processing pipeline - streamlined BoW



## POC Only - Sample Data

In [1]:
import re
import os
import time
import json

import numpy as np
import pandas as pd

import cleanup_module_POC as Cmod
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# load minimally prepared X, y train subsets
deduped_path = os.path.join("..","data","1.2_deduped","tweets","prepared")
X_train = pd.read_csv(os.path.join(deduped_path, "X_train.csv"))
y_train = pd.read_csv(os.path.join(deduped_path, "y_train.csv"))

In [2]:
# sample down considerably to X, y sample subsets
from sklearn.model_selection import train_test_split

X, X_rest, y, y_rest = train_test_split(X_train, y_train, test_size=0.87, random_state=154)

The plan is to forget about the `_rest` datasets and focus on the X, y small subsets, as if they were the entire training data.

In [3]:
print(f'Dataset size: {len(X):0.0f}')
print(f'Target distribution: {sum(y["target"])/len(y):0.3f}')

Dataset size: 101287
Target distribution: 0.530


In [4]:
# keep indices
X.insert(3, 'index', X.index)
X.index = range(len(X))

In [5]:
# transform into arrays
X_array = np.array(X.iloc[:, 2]).ravel()
y_array = y.iloc[:,0].ravel()

In [6]:
preprocess_pipeline = Pipeline([
    ("document_to_wordcount", Cmod.DocumentToWordCounterTransformer()),
    ("wordcount_to_vector", Cmod.WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_array)

In [7]:
X_train_transformed

<101287x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 810743 stored elements in Compressed Sparse Row format>

### Quick models

In [11]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_array, cv=10, verbose=1, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Mean accuracy: 0.6265562542958864


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   15.1s finished


In [12]:
NB_clf = MultinomialNB()
score = cross_val_score(NB_clf, X_train_transformed, y_array, cv=10, verbose=1, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Mean accuracy: 0.6131192129819172


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.4s finished


---